In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate

In [2]:
import os
import requests
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch


In [3]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_compute_dtype=torch.bfloat16,
    bnb_8bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map="auto",
  quantization_config=quant_config
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [7]:
print(len(questions),len(intents))

9 5


In [10]:
from transformers import pipeline
import pandas as pd
import re

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")
prompt = """Generate 5 example user questions about bank loan inquiry chatbot. Use a casual tone, and vary sentence structure.Don't mention anything other than user scenario in your answer."""
result = pipe(prompt, max_new_tokens=200, temperature= 1.0, top_p=0.95)
generated_text = result[0]['generated_text']

# remove prompt
output_only = generated_text.replace(prompt, "").strip()

# question and intent_list
questions = [re.sub(r"^[\s\-]*\d*\.*\s*", "", line) for line in output_only.split("\n") if line.strip()and not line.lower().startswith("here are")]

df = pd.DataFrame({
    "question": questions,
    "intent": "user scenario"  # intent field
})

# Download as CSV file
df.to_csv("synthetic_loan_questions.csv", index=False, encoding="utf-8-sig")

print("Saved CSV file : synthetic_loan_questions.csv")

Device set to use cuda:0


Saved CSV file : synthetic_loan_questions.csv
